In [1]:

import librosa
import soundfile
import os, glob
import numpy as np

In [2]:
# Extract features (mfcc, chroma, mel) from a sound file

def extract_feature(file, mfcc, chroma, mel):
    with soundfile.SoundFile(file) as file:
        X = file.read(dtype="float32")
        sample_rate=file.samplerate
        if chroma:
            stft=np.abs(librosa.stft(X))
        result=np.array([])
        if mfcc:
            mfccs=np.mean(librosa.feature.mfcc(y=X, sr=sample_rate, n_mfcc=40).T, axis=0)
            result=np.hstack((result, mfccs))
        if chroma:
            chroma=np.mean(librosa.feature.chroma_stft(S=stft, sr=sample_rate).T,axis=0)
            result=np.hstack((result, chroma))
        if mel:
            mel=np.mean(librosa.feature.melspectrogram(X, sr=sample_rate).T,axis=0)
            result=np.hstack((result, mel))
        
        return result

In [3]:
# Emotions in the RAVDESS dataset

emotions={
  '01':'neutral',
  '02':'calm',
  '03':'happy',
  '04':'sad',
  '05':'angry',
  '06':'fearful',
  '07':'disgust',
  '08':'surprised'
}

# Emotions to observe
observed_emotions = ['neutral','happy','angry','surprised']

In [4]:

# Load the data and extract features for each sound file

# wav_files = 'C:\\Users\\admin\\Downloads\\speech-emotion-recognition-ravdess-data\\Actor_*\\*.wav'
wav_files = 'C:\\Users\\admin\\Downloads\\sample-ravdess\\Actor_*\\*.wav'

def load_data(test_size=0.2):
    x,y=[],[]
    for file in glob.glob(wav_files):
        file_name=os.path.basename(file)
        emotion=emotions[file_name.split("-")[2]]
        if emotion not in observed_emotions:
            continue
        feature=extract_feature(file, mfcc=True, chroma=True, mel=True)
        x.append(feature)
        y.append(emotion)
    return train_test_split(np.array(x), y, test_size=test_size, random_state=9)


In [7]:
from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score

# Split the dataset
x_train,x_test,y_train,y_test = load_data(test_size=0.25)
